In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
import turicreate as tc

In [2]:
customers = pd.read_csv('data/recommend_1.csv') 
transactions = pd.read_csv('data/trx_data.csv')

In [3]:
customers.head()

,customerId
0,1553
1,20400
2,19750
3,6334
4,27773


In [4]:
transactions.head()

,customerId,products
0,0,20
1,1,2|2|23|68|68|111|29|86|107|152
2,2,111|107|29|11|11|11|33|23
3,3,164|227
4,5,2|2


In [5]:
transactions['products'] = transactions['products'].apply(lambda x: [int(i) for i in x.split('|')])

In [6]:
transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index()

,customerId,0,1,2,3,4,5,6,7,8,9
0,0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2.0,2.0,23.0,68.0,68.0,111.0,29.0,86.0,107.0,152.0


In [7]:
data = pd.melt(transactions.set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})
data['productId'] = data['productId'].astype(np.int64)

In [8]:
data.head()

,customerId,productId,purchase_count
0,0,1,2
1,0,13,1
2,0,19,3
3,0,20,1
4,0,31,2


In [9]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy

In [10]:
data_dummy = create_data_dummy(data)

In [11]:
df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
df_matrix.head()

productId,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
customerId,,,,,,,,,,,,,,,,,,,,,
0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
print(df_matrix_norm.shape)
df_matrix_norm.head()

(24429, 300)


productId,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
customerId,,,,,,,,,,,,,,,,,,,,,
0,NaN,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,0.166667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# create a table for input to the modeling

d = df_matrix_norm.reset_index()
d.index.names = ['scaled_purchase_freq']
data_norm = pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()
print(data_norm.shape)
data_norm.head()

(133585, 3)


,customerId,productId,scaled_purchase_freq
9,9,0,0.133333
25,25,0,0.133333
32,33,0,0.133333
35,36,0,0.133333
43,44,0,0.133333


In [14]:
def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()

In [15]:
train, test = train_test_split(data, test_size = .2)
print(train.shape, test.shape)

((106868, 3), (26717, 3))


In [16]:
# Using turicreate library, we convert dataframe to SFrame - this will be useful in the modeling part

train_data = tc.SFrame(train)
test_data = tc.SFrame(test)

In [17]:
train_data

customerId,productId,purchase_count
16380,193,1
11494,153,1
14779,137,1
306,146,1
22196,52,2
18484,127,2
16739,47,1
11559,44,2
13947,60,3
6706,45,1


In [18]:
test_data

customerId,productId,purchase_count
756,291,1
16653,211,1
11802,236,2
3140,20,1
9808,245,2
4230,230,1
3816,17,2
13880,9,2
5269,194,1
22386,49,1


In [19]:
# We can define a function for this step as follows

def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [20]:
# lets try with both dummy table and scaled/normalized purchase table

train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

In [21]:
# variables to define field names
user_id = 'customerId'
item_id = 'productId'
target = 'purchase_count'
users_to_recommend = list(transactions[user_id])
n_rec = 10 # number of items to recommend
n_display = 30

In [22]:
popularity_model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)

Preparing data set.

Data has 106868 observations with 23311 users and 300 items.

Data prepared in: 0.062606s

106868 observations to process; with 300 unique items.

In [23]:
# Get recommendations for a list of users to recommend (from customers file)
# Printed below is head / top 30 rows for first 3 customers with 10 recommendations each

popularity_recomm = popularity_model.recommend(users=users_to_recommend, k=n_rec)
popularity_recomm.print_rows(n_display)

recommendations finished on 1000/62483 queries. users per second: 270270

recommendations finished on 2000/62483 queries. users per second: 228024

recommendations finished on 3000/62483 queries. users per second: 277804

recommendations finished on 4000/62483 queries. users per second: 309693

recommendations finished on 5000/62483 queries. users per second: 328709

...

recommendations finished on 58000/62483 queries. users per second: 416646

recommendations finished on 59000/62483 queries. users per second: 413099

recommendations finished on 60000/62483 queries. users per second: 408808

recommendations finished on 61000/62483 queries. users per second: 405461

recommendations finished on 62000/62483 queries. users per second: 401783

In [24]:
# Since turicreate is very accessible library, we can define a model selection function as below

def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

In [25]:
# variables to define field names
# constant variables include:
user_id = 'customerId'
item_id = 'productId'
users_to_recommend = list(customers[user_id])
n_rec = 10 # number of items to recommend
n_display = 30 # to print the head / first few rows in a defined dataset

In [26]:
# these variables will change accordingly
name = 'popularity'
target = 'purchase_dummy'
pop_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23317 users and 300 items.

Data prepared in: 0.064701s

106868 observations to process; with 300 unique items.

recommendations finished on 1000/1000 queries. users per second: 121773

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|    1553    |    123    |  1.0  |  1   |
|    1553    |    235    |  1.0  |  2   |
|    1553    |    159    |  1.0  |  3   |
|    1553    |     35    |  1.0  |  4   |
|    1553    |     76    |  1.0  |  5   |
|    1553    |     11    |  1.0  |  6   |
|    1553    |    202    |  1.0  |  7   |
|    1553    |     20    |  1.0  |  8   |
|    1553    |    107    |  1.0  |  9   |
|    1553    |     55    |  1.0  |  10  |
|   20400    |    123    |  1.0  |  1   |
|   20400    |    235    |  1.0  |  2   |
|   20400    |    159    |  1.0  |  3   |
|   20400    |     35    |  1.0  |  4   |
|   20400    |     76    |  1.0  |  5   |
|   20400    |     11    |  1.0  |  6   |
|   20400    |    202    |  1.0  |  7   |
|   20400    |     20    |  1.0  |  8   |
|   20400    |    107    |  1.0  |  9   |
|   20400    |     55    |  1.0  |  10  |
|   19750    |    123    |  1.0  |

In [27]:
name = 'popularity'
target = 'scaled_purchase_freq'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23352 users and 300 items.

Data prepared in: 0.059488s

106868 observations to process; with 300 unique items.

recommendations finished on 1000/1000 queries. users per second: 323102

+------------+-----------+----------------+------+
| customerId | productId |     score      | rank |
+------------+-----------+----------------+------+
|    1553    |    226    | 0.778911564626 |  1   |
|    1553    |    247    | 0.328244274809 |  2   |
|    1553    |    230    | 0.324444444444 |  3   |
|    1553    |    125    | 0.266666666667 |  4   |
|    1553    |    248    | 0.26329787234  |  5   |
|    1553    |    294    |   0.2515625    |  6   |
|    1553    |     72    | 0.231884057971 |  7   |
|    1553    |    276    | 0.227112676056 |  8   |
|    1553    |    165    | 0.223350253807 |  9   |
|    1553    |     83    | 0.220238095238 |  10  |
|   20400    |    226    | 0.778911564626 |  1   |
|   20400    |    247    | 0.328244274809 |  2   |
|   20400    |    230    | 0.324444444444 |  3   |
|   20400    |    125    | 0.266666666667 |  4   |
|   20400    |    248    | 0.26329787234  |  5   |
|   20400    |    294    |   0.2515625    |  6   |
|   20400    |     72    | 0.23

In [28]:
train.groupby(by=item_id)['purchase_count'].mean().sort_values(ascending=False).head(20)

productId
132    3.066667
34     3.015810
37     2.976744
0      2.952427
3      2.863445
248    2.840909
27     2.774194
245    2.697368
110    2.684211
32     2.679612
10     2.640118
230    2.621429
82     2.595556
129    2.541899
58     2.526882
83     2.509434
54     2.498233
252    2.473684
226    2.469388
91     2.432653
Name: purchase_count, dtype: float64

In [29]:
# these variables will change accordingly
name = 'cosine'
target = 'purchase_count'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23311 users and 300 items.

Data prepared in: 0.069935s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 1.05515s

recommendations finished on 1000/1000 queries. users per second: 154536

+------------+-----------+-----------------+------+
| customerId | productId |      score      | rank |
+------------+-----------+-----------------+------+
|    1553    |     2     |  0.118224442005 |  1   |
|    1553    |     1     | 0.0823344111443 |  2   |
|    1553    |     35    | 0.0754737973213 |  3   |
|    1553    |     5     | 0.0741784930229 |  4   |
|    1553    |     33    | 0.0643378734589 |  5   |
|    1553    |     17    | 0.0597451925278 |  6   |
|    1553    |     8     | 0.0491762399673 |  7   |
|    1553    |     41    | 0.0486076831818 |  8   |
|    1553    |     76    | 0.0463675141335 |  9   |
|    1553    |    167    | 0.0459497451782 |  10  |
|   20400    |    284    | 0.0486576557159 |  1   |
|   20400    |     6     |  0.038588643074 |  2   |
|   20400    |    132    | 0.0359700918198 |  3   |
|   20400    |     26    | 0.0357463955879 |  4   |
|   20400    |    198    | 0.0347434878349 |  5   |
|   20400    |    131    | 0.0344877839088 |  6   |
|   20400   

In [30]:
# these variables will change accordingly
name = 'cosine'
target = 'purchase_dummy'
cos_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23317 users and 300 items.

Data prepared in: 0.055762s

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.032532s

recommendations finished on 1000/1000 queries. users per second: 86333.4

In [31]:
name = 'cosine'
target = 'scaled_purchase_freq'
cos_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23352 users and 300 items.

Data prepared in: 0.063192s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.028667s

recommendations finished on 1000/1000 queries. users per second: 194590

In [32]:
# these variables will change accordingly
name = 'pearson'
target = 'purchase_count'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23311 users and 300 items.

Data prepared in: 0.066046s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.054985s

recommendations finished on 1000/1000 queries. users per second: 91083

+------------+-----------+---------------+------+
| customerId | productId |     score     | rank |
+------------+-----------+---------------+------+
|    1553    |    132    | 3.06666666667 |  1   |
|    1553    |     34    | 2.99865378302 |  2   |
|    1553    |     37    | 2.97581589139 |  3   |
|    1553    |     0     | 2.95018876779 |  4   |
|    1553    |     3     | 2.86270859322 |  5   |
|    1553    |    248    | 2.83779164878 |  6   |
|    1553    |     27    | 2.77370169086 |  7   |
|    1553    |    245    | 2.69736842105 |  8   |
|    1553    |     32    | 2.67961165049 |  9   |
|    1553    |    110    | 2.67633323607 |  10  |
|   20400    |    132    | 3.05505580107 |  1   |
|   20400    |     34    | 3.01529421967 |  2   |
|   20400    |     37    |  2.9762148372 |  3   |
|   20400    |     0     | 2.95242718447 |  4   |
|   20400    |     3     |  2.8621127375 |  5   |
|   20400    |    248    | 2.84090909091 |  6   |
|   20400    |     27    | 2.77144911212 |  7   |


In [33]:
# these variables will change accordingly
name = 'pearson'
target = 'purchase_dummy'
pear_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23317 users and 300 items.

Data prepared in: 0.057547s

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.040111s

recommendations finished on 1000/1000 queries. users per second: 144300

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|    1553    |    123    |  0.0  |  1   |
|    1553    |    235    |  0.0  |  2   |
|    1553    |    159    |  0.0  |  3   |
|    1553    |     35    |  0.0  |  4   |
|    1553    |     76    |  0.0  |  5   |
|    1553    |     11    |  0.0  |  6   |
|    1553    |    202    |  0.0  |  7   |
|    1553    |     20    |  0.0  |  8   |
|    1553    |    107    |  0.0  |  9   |
|    1553    |     55    |  0.0  |  10  |
|   20400    |    123    |  0.0  |  1   |
|   20400    |    235    |  0.0  |  2   |
|   20400    |    159    |  0.0  |  3   |
|   20400    |     35    |  0.0  |  4   |
|   20400    |     76    |  0.0  |  5   |
|   20400    |     11    |  0.0  |  6   |
|   20400    |    202    |  0.0  |  7   |
|   20400    |     20    |  0.0  |  8   |
|   20400    |    107    |  0.0  |  9   |
|   20400    |     55    |  0.0  |  10  |
|   19750    |    123    |  0.0  |

In [34]:
name = 'pearson'
target = 'scaled_purchase_freq'
pear_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23352 users and 300 items.

Data prepared in: 0.07185s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.041082s

recommendations finished on 1000/1000 queries. users per second: 85733.9

In [35]:
# create initial callable variables

models_w_counts = [popularity_model, cos, pear]
models_w_dummy = [pop_dummy, cos_dummy, pear_dummy]
models_w_norm = [pop_norm, cos_norm, pear_norm]

names_w_counts = ['Popularity Model on Purchase Counts', 'Cosine Similarity on Purchase Counts', 'Pearson Similarity on Purchase Counts']
names_w_dummy = ['Popularity Model on Purchase Dummy', 'Cosine Similarity on Purchase Dummy', 'Pearson Similarity on Purchase Dummy']
names_w_norm = ['Popularity Model on Scaled Purchase Counts', 'Cosine Similarity on Scaled Purchase Counts', 'Pearson Similarity on Scaled Purchase Counts']

In [36]:
eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)

PROGRESS: Evaluate model Popularity Model on Purchase Counts


recommendations finished on 1000/13793 queries. users per second: 175316

recommendations finished on 2000/13793 queries. users per second: 182565

recommendations finished on 3000/13793 queries. users per second: 186370

recommendations finished on 4000/13793 queries. users per second: 189861

...

recommendations finished on 10000/13793 queries. users per second: 157154

recommendations finished on 11000/13793 queries. users per second: 158868

recommendations finished on 12000/13793 queries. users per second: 160832

recommendations finished on 13000/13793 queries. users per second: 154492


Precision and recall summary statistics by cutoff
+--------+------------------+-------------------+
| cutoff |  mean_precision  |    mean_recall    |
+--------+------------------+-------------------+
|   1    | 0.00130500978757 | 0.000643701256331 |
|   2    | 0.0034800261002  |  0.00378845623374 |
|   3    | 0.00381836197105 |  0.00633722435744 |
|   4    | 0.0078119335895  |  0.0177295366216  |
|   5    | 0.0080475603567  |  0.0226118466877  |
|   6    | 0.00680296768892 |  0.0229743494064  |
|   7    | 0.00631790452714 |  0.0248355688962  |
|   8    | 0.00587254404408 |  0.0263074162445  |
|   9    | 0.00556643063712 |  0.0279378585324  |
|   10   | 0.00535779018343 |  0.0300380439266  |
+--------+------------------+-------------------+
[10 rows x 3 columns]


Overall RMSE: 1.03835316824

Per User RMSE (best)
+------------+-------+------------------+
| customerId | count |       rmse       |
+------------+-------+------------------+
|   13424    |   1   | 4.4408920985e-16 |
+------

In [37]:
eval_dummy = tc.recommender.util.compare_models(test_data_dummy, models_w_dummy, model_names=names_w_dummy)

PROGRESS: Evaluate model Popularity Model on Purchase Dummy


recommendations finished on 1000/13804 queries. users per second: 161943

recommendations finished on 2000/13804 queries. users per second: 179565

recommendations finished on 3000/13804 queries. users per second: 181807

recommendations finished on 4000/13804 queries. users per second: 190042

...

recommendations finished on 10000/13804 queries. users per second: 205524

recommendations finished on 11000/13804 queries. users per second: 206507

recommendations finished on 12000/13804 queries. users per second: 200043

recommendations finished on 13000/13804 queries. users per second: 195345


Precision and recall summary statistics by cutoff
+--------+------------------+------------------+
| cutoff |  mean_precision  |   mean_recall    |
+--------+------------------+------------------+
|   1    | 0.00637496377861 | 0.00292967761817 |
|   2    | 0.00525210084034 | 0.0050043868122  |
|   3    | 0.00521587945523 | 0.00779943277046 |
|   4    | 0.00519776876268 | 0.0106225619116  |
|   5    | 0.00482468849609 | 0.0124753388314  |
|   6    | 0.0046242634985  | 0.0146713321712  |
|   7    | 0.0045742434905  | 0.0169870573119  |
|   8    | 0.00448239640684 | 0.0191411948289  |
|   9    | 0.00457999291671 | 0.0220672790557  |
|   10   | 0.00464358157056 | 0.0247319969805  |
+--------+------------------+------------------+
[10 rows x 3 columns]


Overall RMSE: 1.0

Per User RMSE (best)
+------------+-------+------+
| customerId | count | rmse |
+------------+-------+------+
|    3006    |   1   | 1.0  |
+------------+-------+------+
[1 rows x 3 columns]


Per User RMSE (worst)
+---

In [38]:
eval_norm = tc.recommender.util.compare_models(test_data_norm, models_w_norm, model_names=names_w_norm)

PROGRESS: Evaluate model Popularity Model on Scaled Purchase Counts


recommendations finished on 1000/13859 queries. users per second: 143431

recommendations finished on 2000/13859 queries. users per second: 165975

recommendations finished on 3000/13859 queries. users per second: 174510

recommendations finished on 4000/13859 queries. users per second: 174634

...

recommendations finished on 10000/13859 queries. users per second: 148320

recommendations finished on 11000/13859 queries. users per second: 149130

recommendations finished on 12000/13859 queries. users per second: 148539

recommendations finished on 13000/13859 queries. users per second: 146373


Precision and recall summary statistics by cutoff
+--------+------------------+------------------+
| cutoff |  mean_precision  |   mean_recall    |
+--------+------------------+------------------+
|   1    | 0.00238112417923 |  0.001102859067  |
|   2    | 0.00223681362292 | 0.00202838926953 |
|   3    | 0.00223681362292 | 0.00350825966491 |
|   4    | 0.00214661952522 | 0.00491356960619 |
|   5    | 0.0018183130096  | 0.00520047273601 |
|   6    | 0.00203237366813 | 0.00725329039962 |
|   7    | 0.00299959799202 | 0.0120440572728  |
|   8    | 0.00285915289703 | 0.0129539457877  |
|   9    | 0.00283810760757 | 0.0142735239815  |
|   10   | 0.00275633162566 | 0.0153775856364  |
+--------+------------------+------------------+
[10 rows x 3 columns]


Overall RMSE: 0.134493734496

Per User RMSE (best)
+------------+-------+-------------------+
| customerId | count |        rmse       |
+------------+-------+-------------------+
|    5996    |   1   | 1.56759879118e-06 |
+------------+--

In [39]:
users_to_recommend = list(customers[user_id])

final_model = tc.item_similarity_recommender.create(tc.SFrame(data_dummy), 
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target='purchase_dummy', 
                                            similarity_type='cosine')

recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 133585 observations with 24429 users and 300 items.

Data prepared in: 0.058597s

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.027239s

recommendations finished on 1000/1000 queries. users per second: 37929.1

+------------+-----------+-----------------+------+
| customerId | productId |      score      | rank |
+------------+-----------+-----------------+------+
|    1553    |     1     |  0.103481757641 |  1   |
|    1553    |     2     | 0.0934672474861 |  2   |
|    1553    |     35    | 0.0845762014389 |  3   |
|    1553    |     33    | 0.0668614387512 |  4   |
|    1553    |     61    | 0.0651255607605 |  5   |
|    1553    |     15    | 0.0647641539574 |  6   |
|    1553    |     11    | 0.0546789884567 |  7   |
|    1553    |     5     | 0.0540698170662 |  8   |
|    1553    |     17    | 0.0519999623299 |  9   |
|    1553    |     36    | 0.0504865050316 |  10  |
|   20400    |     26    | 0.0581226944923 |  1   |
|   20400    |     6     | 0.0536174178123 |  2   |
|   20400    |    113    | 0.0531278848648 |  3   |
|   20400    |     1     | 0.0521045923233 |  4   |
|   20400    |     15    | 0.0476838946342 |  5   |
|   20400    |     27    | 0.0446733832359 |  6   |
|   20400   

In [40]:
df_rec = recom.to_dataframe()
print(df_rec.shape)
df_rec.head()

(10000, 4)


,customerId,productId,score,rank
0,1553,1,0.103482,1
1,1553,2,0.093467,2
2,1553,35,0.084576,3
3,1553,33,0.066861,4
4,1553,61,0.065126,5


In [41]:
df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id].transform(lambda x: '|'.join(x.astype(str)))
df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates().sort_values('customerId').set_index('customerId')

In [42]:
def create_output(model, users_to_recommend, n_rec, print_csv=True):
    recomendation = model.recommend(users=users_to_recommend, k=n_rec)
    df_rec = recomendation.to_dataframe()
    df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id] \
        .transform(lambda x: '|'.join(x.astype(str)))
    df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates() \
        .sort_values('customerId').set_index('customerId')
    if print_csv:
        df_output.to_csv('output/option1_recommendation.csv')
        print("An output file can be found in 'output' folder with name 'option1_recommendation.csv'")
    return df_output

In [44]:
df_output = create_output(pear_norm, users_to_recommend, n_rec, print_csv=True)
print(df_output.shape)
df_output.head()

recommendations finished on 1000/1000 queries. users per second: 257998

An output file can be found in 'output' folder with name 'option1_recommendation.csv'
(1000, 1)


,recommendedProducts
customerId,
4,226|247|230|125|248|294|72|276|165|83
11,226|247|230|125|248|294|72|276|165|83
12,226|247|230|125|248|294|72|276|165|83
16,226|247|230|125|248|294|72|276|165|83
21,226|247|230|125|248|294|72|276|165|83


In [45]:
def customer_recomendation(customer_id):
    if customer_id not in df_output.index:
        print('Customer not found.')
        return customer_id
    return df_output.loc[customer_id]

In [46]:
customer_recomendation(4)

recommendedProducts    226|247|230|125|248|294|72|276|165|83
Name: 4, dtype: object

In [47]:
customer_recomendation(21)

recommendedProducts    226|247|230|125|248|294|72|276|165|83
Name: 21, dtype: object